<a href="https://colab.research.google.com/github/cagBRT/timeSeries/blob/main/10g_Multivariate_Multi_Step_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Multiple Input Muti-step Output**

Example of MI-MSOutout<br>
[[10 15 25]<br>
[20 25 45]<br>
[30 35 65]<br>
[40 45 85]<br>
[50 55 105]<br>
[60 65 125]<br>
[70 75 145]<br>
[80 85 165]<br>
[90 95 185]]<br>

If we input:<br>
10, 15<br>
20, 25<br>
30, 35<br>

Output will be: <br>
65<br>
85<br>

**Define the split sequences function**

In [ ]:
def split_sequences(sequences, n_steps_in, n_steps_out):
  X, y = list(), list()
  for i in range(len(sequences)):
    # find the end of this pattern
    end_ix = i + n_steps_in
    out_end_ix = end_ix + n_steps_out-1
    # check if we are beyond the dataset
    if out_end_ix > len(sequences):
      break
    # gather input and output parts of the pattern
    seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1:out_end_ix, -1]
    X.append(seq_x)
    y.append(seq_y)
  return array(X), array(y)

**Import the libraries**

In [ ]:
from numpy import array
from numpy import hstack
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.utils.vis_utils import plot_model

**Prepare the data for the CNN**

In [ ]:
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])

In [ ]:
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))

In [ ]:
dataset = hstack((in_seq1, in_seq2, out_seq))

In [ ]:
n_steps_in, n_steps_out = 3, 2
# convert into input/output
X, y = split_sequences(dataset, n_steps_in, n_steps_out)
n_features=X.shape[2]

In [ ]:
print(X.shape, y.shape)
# summarize the data
for i in range(len(X)):
  print(X[i], y[i])

**Create and train the model**

In [ ]:
model = Sequential()
model.add(Conv1D(24, 2, activation='relu', input_shape=(n_steps_in, n_features))) 
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(18, activation='relu'))
model.add(Dense(n_steps_out))
model.compile(optimizer='adam', loss='mse')

In [ ]:
model.fit(X, y, epochs=200, verbose=0)

In [ ]:
model.summary()

In [ ]:
plot_model(model, show_shapes=True, show_layer_names=True)

**Make a Prediction**<br>
Expecting<br>
185, 205

In [ ]:
# demonstrate prediction
x_input = array([[70, 75], [80, 85], [90, 95]])
x_input = x_input.reshape((1, n_steps_in, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

**Assignment**:<br>
1. Modify the number of neurons and epochs to see if you can improve the model